In [1]:
pip install -e ..

Obtaining file:///home/jovyan/work/Phase5/Assessments/DataJobs
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [1]:
from definitions import ROOT_DIR
print(ROOT_DIR)

/home/jovyan/work/Phase5/Assessments/DataJobs


In [2]:
from importlib import reload
import src.data.mytokenizer
reload(src.data.mytokenizer)

<module 'src.data.mytokenizer' from '/home/jovyan/work/Phase5/Assessments/DataJobs/src/data/mytokenizer.py'>

In [3]:
from src.data import job_database

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
import inspect
from IPython.display import Markdown, display
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

def display_object(obj):
    """Summary of display_object. Get source code for the provided object 
    and display in notebook as markdown.
    """
    source = inspect.getsource(obj)
    wrapped_source = f'```python\n{source}\n```'
    markdown_source = Markdown(wrapped_source)
    display(markdown_source)

In [19]:
query = """
SELECT * from jobs
WHERE title LIKE '%data%'
OR description LIKE '%data%';
"""
db = job_database.JobsDb(ROOT_DIR+'/data/external/jobs.sqlite')
data = db.load_query_as_df(query)
db.close()

In [20]:
descriptions = data['description'].tail(100)

In [24]:
doc = descriptions[19075]

In [25]:
tokenizer = src.data.mytokenizer.LemmaTokenizer()

In [26]:
tokenizer(doc)

['do',
 'you',
 'want',
 'a',
 'new',
 'career',
 'opportunity',
 'that',
 'offer',
 'challenge',
 'and',
 'have',
 'room',
 'for',
 'growth',
 'as',
 'the',
 'world',
 's',
 'large',
 'staffing',
 'and',
 'recruitment',
 'agency',
 'we',
 'can',
 'commit',
 'to',
 'find',
 'you',
 'the',
 'perfect',
 'role',
 'that',
 'give',
 'you',
 'the',
 'opportunity',
 'to',
 'learn',
 'and',
 'grow',
 'in',
 'the',
 'de',
 'moines',
 'market',
 'if',
 'you',
 'be',
 'a',
 'dependable',
 'and',
 'driven',
 'individual',
 'this',
 'may',
 'be',
 'the',
 'perfect',
 'fit',
 'for',
 'you',
 'responsibility',
 'we',
 'be',
 'currently',
 'look',
 'for',
 'a',
 'data',
 'entry',
 'associate',
 'to',
 'work',
 'for',
 'one',
 'of',
 'the',
 'large',
 'financial',
 'company',
 'in',
 'the',
 'de',
 'moines',
 'area',
 'the',
 'ideal',
 'candidate',
 'will',
 'be',
 'a',
 'focused',
 'positive',
 'professional',
 'with',
 'strong',
 'computer',
 'skill',
 'and',
 'attention',
 'to',
 'detail',
 'as',
 '

In [34]:
pipe = Pipeline(
    steps = [
        ('vectorizer', CountVectorizer(
            encoding='utf-8',
            decode_error='ignore',
            strip_accents='unicode',
            lowercase=True,
            preprocessor=None,
            tokenizer=src.data.mytokenizer.LemmaTokenizer(),
            stop_words='english',
            ngram_range=(1,4),
            analyzer='word',
            max_df=0.95,
            min_df=0.05,
            max_features=None
                                      )),
        ('topicModel', LatentDirichletAllocation(n_jobs=-1, verbose=5))
    ]
)

In [35]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
%time pipe.fit(descriptions)

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
CPU times: user 7.88 s, sys: 192 ms, total: 8.07 s
Wall time: 10.9 s


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished


Pipeline(steps=[('vectorizer',
                 CountVectorizer(decode_error='ignore', max_df=0.95,
                                 min_df=0.05, ngram_range=(1, 4),
                                 stop_words='english', strip_accents='unicode',
                                 tokenizer=<src.data.mytokenizer.LemmaTokenizer object at 0x7f483aa5fd00>)),
                ('topicModel',
                 LatentDirichletAllocation(n_jobs=-1, verbose=5))])

In [37]:
%time pipe.transform(descriptions)

CPU times: user 7.41 s, sys: 35.6 ms, total: 7.45 s
Wall time: 7.52 s


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished


array([[4.86697821e-01, 5.61915137e-04, 5.61896494e-04, 1.82648719e-01,
        5.61965028e-04, 5.61922807e-04, 5.61928666e-04, 3.26720148e-01,
        5.61797798e-04, 5.61886132e-04],
       [3.69190184e-01, 6.75852639e-04, 6.75871924e-04, 6.75895850e-04,
        6.75882042e-04, 6.75858763e-04, 6.75821992e-04, 6.25403190e-01,
        6.75675739e-04, 6.75767653e-04],
       [3.84721558e-04, 2.20225740e-01, 7.76696747e-01, 3.84739093e-04,
        3.84646249e-04, 3.84675596e-04, 3.84721286e-04, 3.84692471e-04,
        3.84615418e-04, 3.84701780e-04],
       [2.87304468e-01, 2.80644196e-01, 2.56334593e-01, 4.20289291e-04,
        4.20202791e-04, 4.20265567e-04, 4.20284572e-04, 1.73195276e-01,
        4.20168106e-04, 4.20256739e-04],
       [1.49284805e-03, 1.49275268e-03, 1.69253482e-01, 3.93452278e-01,
        1.49256877e-03, 1.49277066e-03, 1.94895871e-01, 1.49281227e-03,
        1.49253739e-03, 2.33442080e-01],
       [1.59782762e-04, 6.63629522e-01, 3.35092237e-01, 1.59795542e-04,
   

In [31]:
tokens = pipe.named_steps['vectorizer'].get_feature_names()

In [32]:
len(tokens)

1548

In [33]:
tokens

['1',
 '10',
 '10 year',
 '100',
 '12',
 '15',
 '2',
 '2 year',
 '2 year experience',
 '2020',
 '3',
 '3 year',
 '3 year experience',
 '4',
 '40',
 '401',
 '401 k',
 '5',
 '5 year',
 '5 year experience',
 '6',
 '7',
 'ability',
 'ability communicate',
 'ability manage',
 'ability work',
 'ability work effectively',
 'ability work independently',
 'able',
 'accept',
 'access',
 'accommodation',
 'accord',
 'account',
 'accountability',
 'accuracy',
 'accurate',
 'achieve',
 'acquisition',
 'act',
 'action',
 'action employer',
 'action employer qualify',
 'action employer qualify applicant',
 'actionable',
 'active',
 'activity',
 'acumen',
 'addition',
 'additional',
 'address',
 'administer',
 'advance',
 'advance diversity',
 'advanced',
 'affirmative',
 'affirmative action',
 'age',
 'age marital',
 'age marital status',
 'age national',
 'age national origin',
 'agency',
 'agile',
 'ai',
 'aid',
 'air',
 'airflow',
 'algorithm',
 'algorithms',
 'align',
 'alignment',
 'allow',
 'an